In [ ]:
import numpy as np
import pandas as pd
from string import punctuation
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import underthesea
import optuna

In [2]:
pre_df1 = pd.read_csv('../datasets/pre_comments.csv') 

In [3]:
pre_df1

,Name,Link,Comments
0,iPhone 15 Pro Max,https://www.thegioididong.com/dtdd/iphone-15-p...,"['Máy nóng rất tệ ,call cideo như muốn nổ,đang..."
1,iPhone 15 Pro Max,https://www.thegioididong.com/dtdd/iphone-15-p...,"['Tốt', 'Máy mới mua sài mau nóng máy quá.khôn..."
2,iPhone 15 Pro Max,https://www.thegioididong.com/dtdd/iphone-15-p...,"['Rất tuyệt vời', 'Ok', 'Rất Tốt', 'Mọi thứ rấ..."
3,iPhone 15 Pro Max,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,['Mình mới mua trả góp. thủ tục nhanh mà nhân ...
4,iPhone 15 Pro Max,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,"['Thủ tục mua hàng nhanh, đơn giản nhé', 'Sử d..."
...,...,...,...
142,iPhone 14 Plus,https://fptshop.com.vn/dien-thoai/iphone-14-pl...,"['màu xanh đẹp lắm luôn, cầm chắc tay, màn hìn..."
143,iPhone 14 Plus,https://fptshop.com.vn/dien-thoai/iphone-14-pl...,"['máy dùng khá tốt, không có gì để chê, pin cũ..."
144,Samsung Galaxy A34 5G,https://www.thegioididong.com/dtdd/samsung-gal...,['Dùng tạm được.nhưng khi mua thì k đc kèm sạc...
145,Samsung Galaxy A34 5G,https://www.thegioididong.com/dtdd/samsung-gal...,['Mình sài gần năm r kh sao mà gần tuần nay cứ...


In [4]:
pre_df1['Link'] = pre_df1['Link'].str.replace(r'^https://www\.thegioididong\.com/.*', 'TGDD', regex=True)

In [5]:
pre_df1.loc[pre_df1['Link'] != 'TGDD', 'Link'] = 'FPT'

In [6]:
pre_df1

,Name,Link,Comments
0,iPhone 15 Pro Max,TGDD,"['Máy nóng rất tệ ,call cideo như muốn nổ,đang..."
1,iPhone 15 Pro Max,TGDD,"['Tốt', 'Máy mới mua sài mau nóng máy quá.khôn..."
2,iPhone 15 Pro Max,TGDD,"['Rất tuyệt vời', 'Ok', 'Rất Tốt', 'Mọi thứ rấ..."
3,iPhone 15 Pro Max,FPT,['Mình mới mua trả góp. thủ tục nhanh mà nhân ...
4,iPhone 15 Pro Max,FPT,"['Thủ tục mua hàng nhanh, đơn giản nhé', 'Sử d..."
...,...,...,...
142,iPhone 14 Plus,FPT,"['màu xanh đẹp lắm luôn, cầm chắc tay, màn hìn..."
143,iPhone 14 Plus,FPT,"['máy dùng khá tốt, không có gì để chê, pin cũ..."
144,Samsung Galaxy A34 5G,TGDD,['Dùng tạm được.nhưng khi mua thì k đc kèm sạc...
145,Samsung Galaxy A34 5G,TGDD,['Mình sài gần năm r kh sao mà gần tuần nay cứ...


In [6]:
def get_comments(df, name):
    comments = df.loc[df['Name'] == name, 'Comments'].values
    return comments

In [48]:
product_name = get_comments(pre_df1, 'iPhone 15 Pro Max')

In [7]:
import ast

In [8]:
def get_comments(df, name):
    comments = df.loc[df['Name'] == name, 'Comments'].values
    return comments
def list_comments(l_comments):
    all_comments = []
    for i in range(len(l_comments)):
        split_comments = ast.literal_eval(l_comments[i])
        for comment in split_comments:
            all_comments.append(comment)
    return all_comments



In [51]:
all_com = list_comments(product_name)

In [52]:
len(all_com)

294

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class SVCModel():
    def __init__(self, kernel='rbf', C=1.0, gamma='scale', attribute=None):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.svc = None
        self.attribute = attribute

    def fit(self, X, y):
        self.svc = SVC(kernel=self.kernel, C=self.C, gamma=self.gamma, probability=True)
        self.svc.fit(X, y[self.attribute])

    def predict(self, X):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        return self.svc.predict(X)

    def predict_proba(self, X):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        return self.svc.predict_proba(X)
    
    def calculate_accuracy_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            accuracy = accuracy_score(y[self.attribute], y_pred)
            return accuracy
        
        y_pred = self.predict(X)
        accuracy = accuracy_score(y[self.attribute], y_pred)
        return accuracy
    
    def calculate_f1_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")

        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                f1 = f1_score(y[self.attribute], y_pred)
            else:
                f1 = f1_score(y[self.attribute], y_pred, average='micro')
            return f1

        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            f1 = f1_score(y[self.attribute], y_pred)
        else:
            f1 = f1_score(y[self.attribute], y_pred, average='micro')
        return f1
    
    def calculate_precision_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                precision = precision_score(y[self.attribute], y_pred)
            else:
                precision = precision_score(y[self.attribute], y_pred, average='micro')
            return precision
        
        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            precision = precision_score(y[self.attribute], y_pred)
        else:
            precision = precision_score(y[self.attribute], y_pred, average='micro')
        return precision
    
    def calculate_recall_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                recall = recall_score(y[self.attribute], y_pred)
            else:
                recall = recall_score(y[self.attribute], y_pred, average='micro')
            return recall
        
        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            recall = recall_score(y[self.attribute], y_pred)
        else:
            recall = recall_score(y[self.attribute], y_pred, average='micro')
        return recall

In [11]:
#preprocess.py
import regex as re
from underthesea import word_tokenize
import string
import codecs
from pyvi import ViTokenizer


# remove html tags
def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

# unicode stardard
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# diacritic standard
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)

def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True

def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)

def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# remove unnecessary characters
def remove_unnecessary(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ_]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text

# special wordings in reviews (e.g., emoji,...)
def normalize_money(sent):
    return re.sub(r'[0-9]+[.,0-9][k-m-b]', 'giá', sent)

def normalize_hastag(sent):
    return re.sub(r'#+\w+', 'tag', sent)

def normalize_website(sent):
    result = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'website', sent)
    return re.sub(r'\w+(\.(com|vn|me))+((\/+([\.\w\_\-]+)?)+)?', 'website', result)

def nomalize_emoji(sent):
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', sent)

def normalize_elongate(sent):
    patern = r'(.)\1{1,}'
    result = sent
    while(re.search(patern, result) != None):
        repeat_char = re.search(patern, result)
        result = result.replace(repeat_char[0], repeat_char[1])
    return result

def remove_numbers(sent):
    return re.sub(r'[0-9]+', '', sent)

def normalize_acronyms(sent):
    text = sent
    replace_list = {
            "òa": "oà",
        "Òa": "Oà",
        "ÒA": "OÀ",
        "óa": "oá",
        "Óa": "Oá",
        "ÓA": "OÁ",
        "ỏa": "oả",
        "Ỏa": "Oả",
        "ỎA": "OẢ",
        "õa": "oã",
        "Õa": "Oã",
        "ÕA": "OÃ",
        "ọa": "oạ",
        "Ọa": "Oạ",
        "ỌA": "OẠ",
        "òe": "oè",
        "Òe": "Oè",
        "ÒE": "OÈ",
        "óe": "oé",
        "Óe": "Oé",
        "ÓE": "OÉ",
        "ỏe": "oẻ",
        "Ỏe": "Oẻ",
        "ỎE": "OẺ",
        "õe": "oẽ",
        "Õe": "Oẽ",
        "ÕE": "OẼ",
        "ọe": "oẹ",
        "Ọe": "Oẹ",
        "ỌE": "OẸ",
        "ùy": "uỳ",
        "Ùy": "Uỳ",
        "ÙY": "UỲ",
        "úy": "uý",
        "Úy": "Uý",
        "ÚY": "UÝ",
        "ủy": "uỷ",
        "Ủy": "Uỷ",
        "ỦY": "UỶ",
        "ũy": "uỹ",
        "Ũy": "Uỹ",
        "ŨY": "UỸ",
        "ụy": "uỵ",
        "Ụy": "Uỵ",
        "ỤY": "UỴ",
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
            #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
            "👹": "Tiêu cực", "👻": "Tích cực", "💃": "Tích cực",'🤙': ' Tích cực ', '👍': ' Tích cực ',
            "💄": "Tích cực", "💎": "Tích cực", "💩": "Tích cực","😕": "Tiêu cực", "😱": "Tiêu cực", "😸": "Tích cực",
            "😾": "Tiêu cực", "🚫": "Tiêu cực",  "🤬": "Tiêu cực","🧚": "Tích cực", "🧡": "Tích cực",'🐶':' Tích cực ',
            '👎': ' Tiêu cực ', '😣': ' Tiêu cực ','✨': ' Tích cực ', '❣': ' Tích cực ','☀': ' Tích cực ',
            '♥': ' Tích cực ', '🤩': ' Tích cực ', 'like': ' Tích cực ', '💌': ' Tích cực ',
            '🤣': ' Tích cực ', '🖤': ' Tích cực ', '🤤': ' Tích cực ', ':(': ' Tiêu cực ', '😢': ' Tiêu cực ',
            '❤': ' Tích cực ', '😍': ' Tích cực ', '😘': ' Tích cực ', '😪': ' Tiêu cực ', '😊': ' Tích cực ',
            '?': ' ? ', '😁': ' Tích cực ', '💖': ' Tích cực ', '😟': ' Tiêu cực ', '😭': ' Tiêu cực ',
            '💯': ' Tích cực ', '💗': ' Tích cực ', '♡': ' Tích cực ', '💜': ' Tích cực ', '🤗': ' Tích cực ',
            '^^': ' Tích cực ', '😨': ' Tiêu cực ', '☺': ' Tích cực ', '💋': ' Tích cực ', '👌': ' Tích cực ',
            '😖': ' Tiêu cực ', '😀': ' Tích cực ', ':((': ' Tiêu cực ', '😡': ' Tiêu cực ', '😠': ' Tiêu cực ',
            '😒': ' Tiêu cực ', '🙂': ' Tích cực ', '😏': ' Tiêu cực ', '😝': ' Tích cực ', '😄': ' Tích cực ',
            '😙': ' Tích cực ', '😤': ' Tiêu cực ', '😎': ' Tích cực ', '😆': ' Tích cực ', '💚': ' Tích cực ',
            '✌': ' Tích cực ', '💕': ' Tích cực ', '😞': ' Tiêu cực ', '😓': ' Tiêu cực ', '️🆗️': ' Tích cực ',
            '😉': ' Tích cực ', '😂': ' Tích cực ', ':v': '  Tích cực ', '=))': '  Tích cực ', '😋': ' Tích cực ',
            '💓': ' Tích cực ', '😐': ' Tiêu cực ', ':3': ' Tích cực ', '😫': ' Tiêu cực ', '😥': ' Tiêu cực ',
            '😃': ' Tích cực ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' Tích cực ', '🤝': ' Tích cực ', '🎈': ' Tích cực ',
            '😗': ' Tích cực ', '🤔': ' Tiêu cực ', '😑': ' Tiêu cực ', '🔥': ' Tiêu cực ', '🙏': ' Tiêu cực ',
            '🆗': ' Tích cực ', '😻': ' Tích cực ', '💙': ' Tích cực ', '💟': ' Tích cực ',
            '😚': ' Tích cực ', '❌': ' Tiêu cực ', '👏': ' Tích cực ', ';)': ' Tích cực ', '<3': ' Tích cực ',
            '🌝': ' Tích cực ',  '🌷': ' Tích cực ', '🌸': ' Tích cực ', '🌺': ' Tích cực ',
            '🌼': ' Tích cực ', '🍓': ' Tích cực ', '🐅': ' Tích cực ', '🐾': ' Tích cực ', '👉': ' Tích cực ',
            '💐': ' Tích cực ', '💞': ' Tích cực ', '💥': ' Tích cực ', '💪': ' Tích cực ',
            '💰': ' Tích cực ',  '😇': ' Tích cực ', '😛': ' Tích cực ', '😜': ' Tích cực ',
            '🙃': ' Tích cực ', '🤑': ' Tích cực ', '🤪': ' Tích cực ','☹': ' Tiêu cực ',  '💀': ' Tiêu cực ',
            '😔': ' Tiêu cực ', '😧': ' Tiêu cực ', '😩': ' Tiêu cực ', '😰': ' Tiêu cực ', '😳': ' Tiêu cực ',
            '😵': ' Tiêu cực ', '😶': ' Tiêu cực ', '🙁': ' Tiêu cực ',
            #Chuẩn hóa 1 số sentiment words/English words
            ':))': '  Tích cực ', ':)': ' Tích cực ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
            'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
            ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
            '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' Tích cực ',
            'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
            'he he': ' Tích cực ','hehe': ' Tích cực ','hihi': ' Tích cực ', 'haha': ' Tích cực ', 'hjhj': ' Tích cực ',
            ' lol ': ' Tiêu cực ',' cc ': ' Tiêu cực ','cute': u' dễ thương ','huhu': ' Tiêu cực ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
            ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được  ', 'dc': u' được ', 'đk': u' được ',
            'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' Tích cực ', 'store': u' cửa hàng ',
            'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
            'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
            'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
            'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
            'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
            'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
            'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
            'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
            'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
            'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' Tích cực ', 'nv' : 'nhân viên', 'nvien' : 'nhân viên',
            ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ', 'pùn' : 'buồn', 'tuỵt vời' : 'tuyệt vời',
            #dưới 3* quy về 1*, trên 3* quy về 5*
            '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
            'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
            '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ',
            }

    for k, v in replace_list.items():
        text = text.replace(k, v)
    return text

# Từ điển tích cực, tiêu cực, phủ định
def load_sentiment_dicts(path_pos, path_nag, path_not):
    with codecs.open(path_pos, 'r', encoding='UTF-8') as f:
        pos = f.readlines()
    pos_list = [n.strip() for n in pos]

    with codecs.open(path_nag, 'r', encoding='UTF-8') as f:
        nag = f.readlines()
    nag_list = [n.strip() for n in nag]

    with codecs.open(path_not, 'r', encoding='UTF-8') as f:
        not_ = f.readlines()
    not_list = [n.strip() for n in not_]

    return pos_list, nag_list, not_list

# Phân tích tình cảm bằng từ điển được xác định trước
def add_sentiment_features(text, pos_list, nag_list, not_list):
    # chuyen punctuation thành space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    text = ViTokenizer.tokenize(text)
    texts = text.split()
    len_text = len(texts)

    texts = [t.replace('_', ' ') for t in texts]
    for i in range(len_text):
        cp_text = texts[i]
        if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
            numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1

            for j in range(numb_word):
                if texts[i + j + 1] in pos_list:
                    texts[i] = 'notpos'
                    texts[i + j + 1] = ''

                if texts[i + j + 1] in nag_list:
                    texts[i] = 'notnag'
                    texts[i + j + 1] = ''
        else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp Tích cực)
            if cp_text in pos_list:
                texts.append('Tích cực')
            elif cp_text in nag_list:
                texts.append('Tiêu cực')

    text = u' '.join(texts)

    #remove nốt những ký tự thừa thãi
    text = text.replace(u'"', u' ')
    text = text.replace(u'️', u'')
    text = text.replace('🏻','')
    return text

# overall preprocessing
def text_preprocess(document, pos_list, nag_list, not_list):
    #đưa về lower
    document = document.lower()
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    
    # chuẩn hóa các ký tự đặc biệt
    document = normalize_money(document)
    document = normalize_hastag(document)
    document = normalize_website(document)
    document = nomalize_emoji(document)
    document = normalize_elongate(document)
    document = normalize_acronyms(document)
    document = remove_numbers(document)

    # chuẩn hóa cách gõ dấu tiếng việt
    document = standardize_sentence_typing(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = remove_unnecessary(document)
    #
    document = add_sentiment_features(document, pos_list, nag_list, not_list)
    return document.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).replace(' '*4, ' ').replace(' '*3, ' ').replace(' '*2, ' ').strip()

In [12]:
path_pos = '../sentiment_dicts/not.txt'
path_nag = '../sentiment_dicts/nag.txt'
path_not = '../sentiment_dicts/not.txt'
pos_list, nag_list, not_list = load_sentiment_dicts(path_pos, path_nag, path_not)


In [164]:
#vectorizer.py
import numpy as np
from gensim.models import KeyedVectors
from underthesea import word_tokenize
def tokenize_all_reviews(embed_model, dims, reviews, pos_list, nag_list, not_list):
    # split each review into a list of words
    # reviews = [text_preprocess(review, pos_list, nag_list, not_list) for review in reviews]
    # reviews_words = [ word_tokenize(review, format='text') for review in reviews]
    reviews_words = [text_preprocess(review, pos_list, nag_list, not_list) for review in reviews]
    # reviews_words = [word_tokenize(review) for review in reviews_words]
    reviews_words = [reviews_word.split() for reviews_word in reviews_words]

    tokenized_reviews = []
    for review in reviews_words:
        mean_embedding = np.zeros(dims)
        num_words = 0
        for word in review:
            try:
                mean_embedding += embed_model[word]
                num_words += 1
            except:
                continue
        mean_embedding /= num_words
        tokenized_reviews.append(mean_embedding)

    return tokenized_reviews 



In [14]:
from gensim.models import KeyedVectors

In [15]:
w2v = KeyedVectors.load('/Mobile-e-commerce-review-sentiment-classification-main/w2v/w2v_model.model')

In [20]:
pre_df1

,Name,Link,Comments
0,iPhone 15 Pro Max,TGDD,"['Máy nóng rất tệ ,call cideo như muốn nổ,đang..."
1,iPhone 15 Pro Max,TGDD,"['Tốt', 'Máy mới mua sài mau nóng máy quá.khôn..."
2,iPhone 15 Pro Max,TGDD,"['Rất tuyệt vời', 'Ok', 'Rất Tốt', 'Mọi thứ rấ..."
3,iPhone 15 Pro Max,FPT,['Mình mới mua trả góp. thủ tục nhanh mà nhân ...
4,iPhone 15 Pro Max,FPT,"['Thủ tục mua hàng nhanh, đơn giản nhé', 'Sử d..."
...,...,...,...
142,iPhone 14 Plus,FPT,"['màu xanh đẹp lắm luôn, cầm chắc tay, màn hìn..."
143,iPhone 14 Plus,FPT,"['máy dùng khá tốt, không có gì để chê, pin cũ..."
144,Samsung Galaxy A34 5G,TGDD,['Dùng tạm được.nhưng khi mua thì k đc kèm sạc...
145,Samsung Galaxy A34 5G,TGDD,['Mình sài gần năm r kh sao mà gần tuần nay cứ...


In [165]:
def get_comments(df, name, company):
    comments = df.loc[(df['Name'] == name) & (df['Link'] == company), 'Comments'].values
    return comments
def list_comments(l_comments):
    all_comments = []
    for i in range(len(l_comments)):
        split_comments = ast.literal_eval(l_comments[i])
        for comment in split_comments:
            all_comments.append(comment)
    return all_comments



In [805]:
product_name = get_comments(pre_df1, 'Samsung Galaxy A34 5G', 'FPT')
product_name = list_comments(product_name)

In [806]:
vec_word = tokenize_all_reviews(w2v, 300, product_name, pos_list, nag_list, not_list)

In [807]:
vec_word_np = np.array(vec_word)

In [808]:
vec_word_np.shape

(45, 300)

In [795]:
nan_mask = np.isnan(vec_word_np)

rows_with_nan = nan_mask.any(axis=1)

cleaned_array = vec_word_np[~rows_with_nan]



In [799]:
cleaned_array.shape

(342, 300)

In [800]:
vec_word = cleaned_array

In [798]:
len(product_name)

345

In [29]:
import joblib


In [396]:
#load model
# svm_Pin = joblib.load('/home/trnmah/CNPM66/Mobile-e-commerce-review-sentiment-classification-main/pretrained_svm/bestModelPin')
# svm_Service = joblib.load('/home/trnmah/CNPM66/Mobile-e-commerce-review-sentiment-classification-main/pretrained_svm/bestModelService')
# svm_General = joblib.load('/home/trnmah/CNPM66/Mobile-e-commerce-review-sentiment-classification-main/pretrained_svm/bestModelGeneral')
# svm_Others = joblib.load('/home/trnmah/CNPM66/Mobile-e-commerce-review-sentiment-classification-main/pretrained_svm/bestModelOthers')

svm_SPin = joblib.load('/home/trnmah/CNPM66/Mobile-e-commerce-review-sentiment-classification-main/pretrained_svm/bestModelSPin')
svm_SSer = joblib.load('/home/trnmah/CNPM66/Mobile-e-commerce-review-sentiment-classification-main/pretrained_svm/bestModelSSer')
svm_SGeneral = joblib.load('/home/trnmah/CNPM66/Mobile-e-commerce-review-sentiment-classification-main/pretrained_svm/bestModelSGeneral')
svm_SOth = joblib.load('/home/trnmah/CNPM66/Mobile-e-commerce-review-sentiment-classification-main/pretrained_svm/bestModelSOth')

/home/trnmah/mambaforge/envs/practic1/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [137]:
# y_pred_Pin = svm_Pin.predict(vec_word_np)
# y_pred_Service = svm_Service.predict(vec_word_np)
# y_pred_General = svm_General.predict(vec_word_np)
# y_pred_Others = svm_Others.predict(vec_word_np)

# y_pred_SPin = svm_SPin.predict_proba(vec_word)
# y_pred_SSer = svm_SSer.predict_proba(vec_word)
# y_pred_SGeneral = svm_SGeneral.predict_proba(vec_word)
# y_pred_SOth = svm_SOth.predict_proba(vec_word)


In [809]:

y_pred_SPin = svm_SPin.predict_proba(vec_word)
y_pred_SSer = svm_SSer.predict_proba(vec_word)
y_pred_SGeneral = svm_SGeneral.predict_proba(vec_word)
y_pred_SOth = svm_SOth.predict_proba(vec_word)

In [810]:
# print(y_pred_Pin.mean(axis=0))
# print(y_pred_Service.mean(axis=0))
# print(y_pred_General.mean(axis=0))
# print(y_pred_Others.mean(axis=0))
print(y_pred_SPin.mean(axis=0))
print(y_pred_SSer.mean(axis=0))
print(y_pred_SGeneral.mean(axis=0))
print(y_pred_SOth.mean(axis=0))


[0.04756708 0.7865609  0.16587201]
[0.0428561  0.50355833 0.45358557]
[0.04055891 0.54013136 0.41930973]
[0.07349256 0.30205295 0.62445449]
